In [12]:
import numpy as np
import torch
import psutil
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import time
import matplotlib.pyplot as plt
import random
import sys

In [13]:
sys.path.insert(1, '/home/ubuntu/Intents/Intents-Analysis/Analysis')

In [14]:
from get_vocab import load_data, get_vocab

In [15]:
#Check if cuda is available
cuda = torch.cuda.is_available()
print('CUDA is', cuda)

num_workers = 8 if cuda else 0

print(num_workers)

CUDA is True
8


In [16]:
print(psutil.virtual_memory())

svmem(total=16481628160, available=15663706112, percent=5.0, used=524664832, free=14307622912, active=1249918976, inactive=679854080, buffers=129814528, cached=1519525888, shared=819200, slab=141631488)


In [17]:
#Defining constants and labels
max_sent_len = {'english': 247, 'hindi': 265, 'gujarati': 283, 'bengali': 295, 'marathi': 307}
intent_labels = {'movie-tickets':0, 'auto-repair':1, 'restaurant-table':2, 'pizza-ordering':3, 'uber-lyft':4, 'coffee-ordering':5}
language = 'english'

#Loading data
data_file = '../Analysis/Labels/TaskMaster/data_taskmaster_' + language + '.pkl'
train_file = '../Analysis/Labels/TaskMaster/taskmaster_training_' + language + '.pkl'
test_file = '../Analysis/Labels/TaskMaster/taskmaster_testing_' + language + '.pkl'

train_data = load_data(train_file)
test_data = load_data(test_file)
vocab, _ = get_vocab(1, data_file)

#creating vocabulary dictionary
my_vocab = {}
for i, phone in enumerate(vocab):
    my_vocab[phone] = i

In [18]:
def create_input_pad_end(utterance, my_vocab, max_len):
    '''
    Pad sentence at the end with maximum length with index max_len 
    '''
    input_vector = []
    for ipa in utterance:
        input_vector.append(my_vocab[ipa])
    
    for i in range(max_len - len(utterance)):
        input_vector.append(len(my_vocab))
        
    return input_vector
    
    

In [91]:
def create_input_pad_start(utterance, my_vocab, max_len):
    '''
    Pad sentence at the end with maximum length with index max_len 
    '''
    input_vector = []
    for ipa in utterance:
        input_vector.append(my_vocab[ipa])
    
    for i in range(max_len - len(utterance)):
        input_vector.insert(0,len(my_vocab))
        
    return input_vector
    

In [92]:
class MyDataset(Dataset):
    def __init__(self, data, my_vocab, intent_labels, max_len, train = True):
        self.all_data = []
        for intent in data:
            for utterance in data[intent]:
                input_vector = create_input_pad_end(utterance,my_vocab, max_len)
                self.all_data.append([torch.from_numpy(np.array(input_vector)).long(), intent_labels[intent]])
        
        if train:
            random.shuffle(self.all_data)
            
    def __len__(self):
        return len(self.all_data)

    def __getitem__(self,index):

        
        return self.all_data[index][0], self.all_data[index][1]

In [93]:
train_dataset = MyDataset(train_data, my_vocab, intent_labels, max_sent_len[language], train=True)
train_loader_args = dict(shuffle=True, batch_size=128, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=True, batch_size=32)
train_loader = DataLoader(train_dataset, **train_loader_args)

test_dataset = MyDataset(test_data, my_vocab, intent_labels, max_sent_len[language], train=False)
test_loader_args = dict(shuffle=False, batch_size=128, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=False, batch_size=1)
test_loader = DataLoader(test_dataset, **test_loader_args)

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/multiprocessing/queues.py", line 230, in _feed
    close()
  File "/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/

In [94]:
len(train_loader),len(test_loader)

(23, 3)

In [95]:
for data in train_loader:
    print(data[0].shape)
    break

torch.Size([128, 247])


In [101]:
class MyRNN_Model(nn.Module):
    def __init__(self, vocab_size):
        super(MyRNN_Model, self).__init__()

        self.emb_dim = 10
        self.lstm_hidden = 64
        
        self.emb = nn.Embedding(vocab_size, self.emb_dim, padding_idx = vocab_size - 1)
        self.rnn = nn.LSTM(self.emb_dim, self.lstm_hidden, num_layers = 1, batch_first = True)
        self.fc = nn.Linear(self.lstm_hidden, 6)

    def init_rnn_hid(self):
        return torch.zeros(1, 1, self.rnn_hid_dim)

    def forward(self, x):
        x = self.emb(x) # (seq_len, rnn_in_dim)
        x, _ = self.rnn(x)
        x = self.fc(x[:,-1,:])
        return x

In [102]:
model = MyRNN_Model(len(my_vocab) + 1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
device = torch.device("cuda" if cuda else "cpu")
model.to(device)
print(model)
print(optimizer)

MyRNN_Model(
  (emb): Embedding(49, 10, padding_idx=48)
  (rnn): LSTM(10, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=6, bias=True)
)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)


In [103]:
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    total_predictions = 0.0
    correct_predictions = 0.0
    
    start_time = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):   
        optimizer.zero_grad()   # .backward() accumulates gradients
        data = data.to(device)
        target = target.to(device) # all data & model on same device

        outputs = model(data)
        loss = criterion(outputs, target)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += target.size(0)
        correct_predictions += (predicted == target).sum().item()
    
            
    end_time = time.time()
    
    acc = (correct_predictions/total_predictions)*100.0
    running_loss /= len(train_loader)
    print('Training Loss: ', running_loss, 'Time: ',end_time - start_time, 's')  
    print('Training Accuracy: ', acc, '%')
    return running_loss

In [104]:
def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0

        for batch_idx, (data, target) in enumerate(test_loader):   
            data = data.to(device)
            target = target.to(device)

            outputs = model(data)

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()

            loss = criterion(outputs, target).detach()
            running_loss += loss.item()


        running_loss /= len(test_loader)
        acc = (correct_predictions/total_predictions)*100.0
        print('Testing Loss: ', running_loss)
        print('Testing Accuracy: ', acc, '%')
        return running_loss, acc

In [105]:
Train_loss = []
Test_loss = []
Test_acc = []
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, cooldown=3)
 
for i in range(30):
    train_loss = train_epoch(model, train_loader, criterion, optimizer)
    test_loss, test_acc = test_model(model, test_loader, criterion)
    Train_loss.append(train_loss)
    Test_loss.append(test_loss)
    Test_acc.append(test_acc)

    #scheduler.step(test_acc)
    for param_group in optimizer.param_groups:
        print('Learning rate:', param_group['lr'])
    

    print('='*20)

Training Loss:  1.7644311811613 Time:  0.6188681125640869 s
Training Accuracy:  22.460074753652734 %
Testing Loss:  1.88696293036143
Testing Accuracy:  16.666666666666664 %
Learning rate: 0.001
Training Loss:  1.7412151979363484 Time:  0.49326205253601074 s
Training Accuracy:  23.887189942235814 %
Testing Loss:  1.961968461672465
Testing Accuracy:  16.666666666666664 %
Learning rate: 0.001
Training Loss:  1.7397717600283416 Time:  0.4608328342437744 s
Training Accuracy:  24.02310567448182 %
Testing Loss:  1.9833447933197021
Testing Accuracy:  16.666666666666664 %
Learning rate: 0.001
Training Loss:  1.7385932207107544 Time:  0.4586319923400879 s
Training Accuracy:  24.02310567448182 %
Testing Loss:  1.99408757686615
Testing Accuracy:  16.666666666666664 %
Learning rate: 0.001
Training Loss:  1.7384285408517588 Time:  0.45691776275634766 s
Training Accuracy:  24.02310567448182 %
Testing Loss:  1.9911902745564778
Testing Accuracy:  16.666666666666664 %
Learning rate: 0.001
Training Loss:

KeyboardInterrupt: 

In [9]:
torch.from_numpy(np.array([1,2,3,])).float()

tensor([1., 2., 3.])